In [4]:
import math
from dataclasses import dataclass
from typing import List, Optional, Tuple


# SortDet + SortDetRev (gespiegelte Variante) 
class SortDet:
    def __init__(self, A, start, end, label="SortDet"):
        self.A = A
        self.start = start
        self.end = end
        self.n = end - start
        self.label = label

        #Randfall triviale Struktur 
        if self.n <= 1:
            self.subarrays = [(start, end)]
            self.subarray_fill = [0]
            self.used_intervals = [set()]
            self.N1 = 1
            self.interval_len = 1
            return
                   
        self.N1 = max(1, math.isqrt(self.n)) #Anzahl Werteintervalle 
        self.interval_len = 1 / self.N1 #Länge Werteintervall
        self.N2 = 2 * self.N1 #Anzahl Buckets 

        #Baue Bucket-Struktur gleichmäßig, sodass alle Buckets gleich groß (+/- 1) 
        base_size = self.n // self.N2
        remainder = self.n % self.N2
        s = start
        self.subarrays = []
        for i in range(self.N2):
            e = s + base_size + (1 if i < remainder else 0)
            self.subarrays.append((s, e))
            s = e

        #subarray_fill[j] = wie viele Zellen im j-ten Subarray schon belegt sind
        self.subarray_fill = [0] * self.N2
        #used_intervals[j] = Menge der Intervall-IDs, die bereits in diesem Subarray landen
        self.used_intervals = [set() for _ in range(self.N2)]

    #Mappt Wert x ∈ [0,1] auf einen Intervallindex (0..N1-1)
    def get_interval_index(self, x: float) -> int:
        return min(int(x / self.interval_len), self.N1 - 1)

    def insert(self, x: float) -> bool:
        i = self.get_interval_index(x)
        # Fall 1: Bucket hat noch Platz und kennt das Intervall schon -> x einfügen
        for j, (s, e) in enumerate(self.subarrays):
            if self.subarray_fill[j] < (e - s) and i in self.used_intervals[j]:
                pos = s + self.subarray_fill[j]
                self.A[pos] = x
                self.subarray_fill[j] += 1
                return True
        # Fall 2: Leeres Bucket gefunden -> ordne Intervall diesem zu und x einfügen
        for j, (s, e) in enumerate(self.subarrays):
            if (e - s) > 0 and self.subarray_fill[j] == 0:
                self.A[s] = x
                self.subarray_fill[j] = 1
                self.used_intervals[j].add(i)
                return True
        # Fall 3: Kein passendes oder leeres Bucket -> fülle irgendwo mit Restkapazität, "Rekursion"
        for j, (s, e) in enumerate(self.subarrays):
            if self.subarray_fill[j] < (e - s):
                pos = s + self.subarray_fill[j]
                self.A[pos] = x
                self.subarray_fill[j] += 1
                self.used_intervals[j].add(i)
                return True
        return False


# SortDetRev: gespiegelte Version, Insert-Logik nahezu identisch mit SortDet 
# Unterschied: Buckets werden von rechts nach links gefüllt (statt links nach rechts)
class SortDetRev(SortDet):
    # gespiegelt (rechts -> links)
    def insert(self, x: float) -> bool:
        i = self.get_interval_index(x)
        # Fall 1
        for j, (s, e) in enumerate(self.subarrays):
            if self.subarray_fill[j] < (e - s) and i in self.used_intervals[j]:
                pos = e - 1 - self.subarray_fill[j]
                self.A[pos] = x
                self.subarray_fill[j] += 1
                return True
        # Fall 2
        for j, (s, e) in enumerate(self.subarrays):
            if (e - s) > 0 and self.subarray_fill[j] == 0:
                pos = e - 1
                self.A[pos] = x
                self.subarray_fill[j] = 1
                self.used_intervals[j].add(i)
                return True
        # Fall 3
        for j, (s, e) in enumerate(self.subarrays):
            if self.subarray_fill[j] < (e - s):
                pos = e - 1 - self.subarray_fill[j]
                self.A[pos] = x
                self.subarray_fill[j] += 1
                self.used_intervals[j].add(i)
                return True
        return False

